##Convolutional Neural Networks

Convolutions extract features by applying filters to the input 

In [1]:
import tensorflow_datasets as tfds

dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=Truem)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete1YX4YE/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete1YX4YE/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [3]:
#Get the tokenizer 
tokenizer = info.features['text'].encoder

BUFFER_SIZE = 10000
BATCH_SIZE = 256

#Get the train and test splits 
train_data, test_data = dataset['train'], dataset['test']

#Shuffle the training data
train_dataset = train_data.shuffle(BUFFER_SIZE)

#Batch and pad the datasets to the maximum length of the sequences
train_dataset = train_dataset.padded_batch(BATCH_SIZE)
test_dataset = test_data.padded_batch(BATCH_SIZE)



##Build the model

For temporal data such as text sequences, you will use Conv1D instead of Conv2D. You will also append a pooling layer to reduce the output of the convolutional layer 

Use GlobalMaxPooling1D to get the max value across the time dimension

In [4]:
import tensorflow as tf
import numpy as np

# Hyper parameters
batch_size = 1
timesteps = 20
features = 20
filters = 128
kernel_size = 5

print(f'batch_size: {batch_size}')
print(f'timesteps (sequence length): {timesteps}')
print(f'features (embedding size): {features}')
print(f'filters: {filters}')
print(f'kernel_size: {kernel_size}')

#Define array input with random values 
random_input = np.random.rand(batch_size, timesteps, features)
print(f'Shape of input array: {random_input.shape}')

#Pass array to convolution layer and inspect output shape 
conv1d = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')
result = conv1d(random_input)
print(f'Shape of conv1e output: {result.shape}')

#Pass array to max pooling layer and inspect output shape
gmp = tf.keras.layers.GlobalMaxPooling1D()
result = gmp(result)
print(f'Shape of global max pooling output: {result.shape}')

batch_size: 1
timesteps (sequence length): 20
features (embedding size): 20
filters: 128
kernel_size: 5
Shape of input array: (1, 20, 20)
Shape of conv1e output: (1, 16, 128)
Shape of global max pooling output: (1, 128)


You can add the convolutiona and pooling layers after the embedding layers by appending them to the model 

In [6]:
import tensorflow as tf

#Hyper parameters 
embedding_dim = 64
filters = 128
kernel_size = 5
dense_dim = 64

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
    tf.keras.layers.Conv1D(filters = filters, kernel_size = kernel_size, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          523840    
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         41088     
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 573,249
Trainable params: 573,249
Non-trainable params: 0
__________________________________________________

In [8]:
# Set the training parameters
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
NUM_EPOCHS = 10

#Train the model 
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/10
98/98 [==============================] - 206s 2s/step - loss: 0.6534 - accuracy: 0.6066 - val_loss: 0.4922 - val_accuracy: 0.8058
Epoch 2/10
98/98 [==============================] - 207s 2s/step - loss: 0.3329 - accuracy: 0.8760 - val_loss: 0.3246 - val_accuracy: 0.8721
Epoch 3/10
98/98 [==============================] - 209s 2s/step - loss: 0.2220 - accuracy: 0.9199 - val_loss: 0.3114 - val_accuracy: 0.8792
Epoch 4/10
98/98 [==============================] - 208s 2s/step - loss: 0.1784 - accuracy: 0.9365 - val_loss: 0.3239 - val_accuracy: 0.8754
Epoch 5/10
98/98 [==============================] - 207s 2s/step - loss: 0.1501 - accuracy: 0.9474 - val_loss: 0.3526 - val_accuracy: 0.8702
Epoch 6/10
